In [1]:
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import numpy as np

In [2]:
driver = webdriver.Chrome()
driver.get('http://school.gter.net/search/countries.html?countrieid=1')
html = driver.page_source
soup = BeautifulSoup(html, 'lxml') 

In [3]:
print(soup.prettify())

<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   美国 地区搜索 寄托天下出国留学网
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="美国留学,加拿大留学,香港留学,新加坡留学,英国留学,欧洲留学, 留学经验分享,DIY留学,留学申请,留学流程,留学费用,出国留学,留学论坛, 留学网站,留学考试,GRE,TOEFL,IBT,GMAT,IELTS,SAT,VISA,文书,签证" name="Keywords"/>
  <meta content="提供最有用的出国留学资讯和最热心的留学交流论坛。在BBS上，你可以咨询签证，面试，机经，offer，奖学金，名校专业等，也可以分享雅思、托福、GRE的学习心得。无论你留学在美国、加拿大、英国还是澳洲，都能在留学论坛上找到寄托情感的归宿。" name="Description"/>
  <base href="http://www.gter.net/"/>
  <link href="image/gter/header.css?v=OIdPT" rel="stylesheet" type="text/css"/>
  <link href="image/gter/css/theme.css?v=OIdPT" rel="stylesheet" type="text/css"/>
  <link href="index.html?m=index&amp;c=combine&amp;t=c&amp;b=[TPL_DIR]&amp;fs=school.i&amp;style=gter" rel="stylesheet" type="text/css"/>
  <script language="javascript" type="text/javascript">
   var site_url='http://school.gter.net/',imgpath='image/',charset='utf-8';
  </script>
  <script src="image/js/common.js?v=

In [73]:
def strip_chinese(string, loc=0):
    return re.sub(re.findall('\w+',string)[loc], '', string).strip()

In [112]:
def extract_school_info(school):
    info = {}
    pieces = school.find_all('td')
    info['name'] = strip_chinese(school.td.text)
    info['link'] = school.td.a['href']    
    info['location'] = strip_chinese(pieces[1].text)
    try:
        info['tuition'] = int(strip_chinese(pieces[2].text,1))
    except:
        info['tuition'] = np.NaN
    info['rating_overall'] = float(pieces[3].text[:-1])
    info['application_count'] = int(pieces[4].text)
    info['accepted_count'] = int(pieces[5].text)
    info['review_count'] = int(pieces[6].text)
    return(info)
#extract_school_info(school)

In [104]:
# scraping first page
tables = soup.find_all('table')[5]
schools = tables.find_all('tr')[1:]
school_info = []
for school in schools:
    school_info.append(extract_school_info(school))
print(school_info)

[{'name': 'Columbia University', 'link': 'http://school.gter.net/school_7.html', 'location': 'New York New York', 'tuition': 59306, 'rating_overall': 8.4, 'application_count': 1304, 'accepted_count': 1683, 'review_count': 3}, {'name': 'Dartmouth College', 'link': 'http://school.gter.net/school_21.html', 'location': 'New Hampshire Hanover', 'tuition': 59262, 'rating_overall': 8.2, 'application_count': 90, 'accepted_count': 64, 'review_count': 0}, {'name': 'Bard College', 'link': 'http://school.gter.net/school_565.html', 'location': 'New York Annandale-On-Hudson', 'tuition': 59232, 'rating_overall': 0.0, 'application_count': 2, 'accepted_count': 2, 'review_count': 0}, {'name': 'University of Chicago', 'link': 'http://school.gter.net/school_15.html', 'location': 'Illinois Chicago', 'tuition': 58458, 'rating_overall': 7.9, 'application_count': 482, 'accepted_count': 455, 'review_count': 0}, {'name': 'Washington University in St Louis', 'link': 'http://school.gter.net/school_26.html', 'loca

In [113]:
# scrape subsequent pages
for page_number in range(2,27):
    driver.get('http://school.gter.net/search/countries.html?countrieid=1&page='+str(page_number))
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml') 
    tables = soup.find_all('table')[5]
    schools = tables.find_all('tr')[1:]
    for school in schools:
        school_info.append(extract_school_info(school))
    time.sleep(np.random.lognormal(0.75) + 0.2)

In [125]:
# convert list of dictionaries to data frame
import pandas as pd
Gter = pd.DataFrame(school_info)
Gter_tmp = Gter.drop_duplicates('name')

In [127]:
Gter = Gter_tmp
Gter.to_csv('Data/Gter.csv', index = False)